In [1]:
import igraph as ig
import numpy as np
import ensembleclustering as CE
from sklearn.metrics import adjusted_mutual_info_score

In [2]:
#%pip install git+https://github.com/mful/ClusterEnsembles.git

In [3]:
# Load Zachary's Karate Club Graph
z = ig.Graph.Famous("Zachary")

# Ground-truth communities (predefined two-group split)
ground_truth = [0,0,0,0,0,0,0,0,1,1,
                0,0,0,0,1,1,0,0,1,0,
                1,0,1,1,1,1,1,1,1,1,
                1,1,1,1]

def compute_ami(partition, true_labels):
    return adjusted_mutual_info_score(true_labels, partition.membership)

In [5]:
# --- 1) Louvain ---
louvain_partition = z.community_multilevel()
ami_louvain = compute_ami(louvain_partition, ground_truth)

# --- 2) Leiden ---
leiden_partition = z.community_leiden(objective_function='modularity')
ami_leiden = compute_ami(leiden_partition, ground_truth)

# --- 3) Infomap ---
infomap_partition = z.community_infomap()
ami_infomap = compute_ami(infomap_partition, ground_truth)

# --- 4) Label Propagation ---
lp_partition = z.community_label_propagation()
ami_lp = compute_ami(lp_partition, ground_truth)

# --- 5) Girvan-Newman (Edge Betweenness) ---
gn_hierarchy = z.community_edge_betweenness()
gn_partition = gn_hierarchy.as_clustering(n=2)
ami_gn = compute_ami(gn_partition, ground_truth)

# --- 6) CNM (Clauset-Newman-Moore, "fastgreedy") ---
cnm_hierarchy = z.community_fastgreedy()
cnm_partition = cnm_hierarchy.as_clustering()
ami_cnm = compute_ami(cnm_partition, ground_truth)

# --- 7) ECG using ClusterEnsembles ---
# Generate multiple base clusterings
labels_louvain = louvain_partition.membership
labels_infomap = infomap_partition.membership
labels_lp = lp_partition.membership

# Combine base clusterings
base_clusterings = np.array([labels_louvain, labels_infomap, labels_lp])

# Perform ensemble clustering
consensus_labels = CE.cluster_ensembles(base_clusterings, verbose=True, nclass=5)

# Compute AMI for ECG
ami_ecg = adjusted_mutual_info_score(ground_truth, consensus_labels)

Cluster Ensembles
    - number of classes: 5
    - solver: hbgf
    - length of base clustering labels: 34
    - number of base clusters: 3


In [6]:
# Print Results
print("ECG AMI:           ", round(ami_ecg, 3))
print("Leiden AMI:        ", round(ami_leiden, 3))
print("Louvain AMI:       ", round(ami_louvain, 3))
print("Infomap AMI:       ", round(ami_infomap, 3))
print("LabelProp AMI:     ", round(ami_lp, 3))
print("Girvan-Newman AMI: ", round(ami_gn, 3))
print("CNM AMI:           ", round(ami_cnm, 3))

ECG AMI:            0.433
Leiden AMI:         0.671
Louvain AMI:        0.565
Infomap AMI:        0.687
LabelProp AMI:      0.708
Girvan-Newman AMI:  0.833
CNM AMI:            0.681
